block 1

In [1]:
!pip install pyspark

In [2]:
import numpy as np
import pyspark
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf


In [3]:
!hdfs dfsadmin -safemode leave


2023-12-19 20:06:28 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Safe mode is OFF


In [4]:
conf = SparkConf().set("spark.executor.instances", "2").set("spark.executor.cores", "1").set("spark.executor.memory", "1g")
spark = SparkSession.builder.master("yarn").appName("shabes_spark").config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/19 20:06:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/19 20:06:34 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [5]:
spark

In [12]:
!hdfs dfs -put ml-latest-small .


2023-12-19 18:59:02 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
!hdfs dfs -ls .

2023-12-19 18:59:14 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 7 items
drwxr-xr-x   - root supergroup          0 2023-12-19 18:52 .sparkStaging
drwxr-xr-x   - root supergroup          0 2023-12-06 06:40 Hadoop_Result_EpisodeAll.txt
drwxr-xr-x   - root supergroup          0 2023-12-06 06:31 Hadoop_Result_SW_EpisodeIV.txt
drwxr-xr-x   - root supergroup          0 2023-12-06 06:32 Hadoop_Result_SW_EpisodeV.txt
drwxr-xr-x   - root supergroup          0 2023-12-06 06:33 Hadoop_Result_SW_EpisodeVI.txt
drwxr-xr-x   - root supergroup          0 2023-12-19 18:59 ml-latest-small
drwxr-xr-x   - root supergroup          0 2023-12-06 06:28 tmp


In [13]:
ratings_schema = StructType(fields=[
    StructField("userId", IntegerType()),
    StructField("movieId", IntegerType()),
    StructField("rating", DoubleType()),
    StructField("timestamp", LongType()),
])

tags_schema = StructType(fields=[
    StructField("userId", IntegerType()),
    StructField("movieId", IntegerType()),
    StructField("tag", StringType()),
    StructField("timestamp", LongType()),
])

In [14]:
ratings = spark.read.format("csv").option("header", "True").schema(ratings_schema).load("ml-latest-small/ratings.csv")
tags = spark.read.format("csv").option("header", "True").schema(tags_schema).load("ml-latest-small/tags.csv")

In [15]:
display(ratings.count())
display(tags.count())

100836

3683

block 2

task 1

In [16]:
print(ratings.groupby("movieId").count().count(), ' unique films')
print(ratings.groupby("userId").count().count(), ' unique users')

9724  unique films
610  unique users


task 2

In [17]:
import pyspark.sql.functions as functions

In [19]:
print(ratings.filter(functions.col("rating") >= 4.0).count(), ' films with more than 4.0 rating')

48580  films with more than 4.0 rating


task 3

In [41]:
print('Top 100 films')
ratings.drop('userId', 'timestamp').groupBy('movieId').agg({"rating": "avg"}).orderBy('avg(rating)', ascending=False).show(100)




Top 100 films


+-------+-----------+
|movieId|avg(rating)|
+-------+-----------+
|  80124|        5.0|
|  74226|        5.0|
| 175387|        5.0|
|    148|        5.0|
| 158027|        5.0|
|  67618|        5.0|
|  42556|        5.0|
| 142444|        5.0|
|  44943|        5.0|
|  27523|        5.0|
| 113829|        5.0|
|   3073|        5.0|
| 152711|        5.0|
| 132333|        5.0|
|     53|        5.0|
|  27704|        5.0|
| 147300|        5.0|
| 187717|        5.0|
|    633|        5.0|
|   8238|        5.0|
|    876|        5.0|
| 120138|        5.0|
|  53578|        5.0|
| 163072|        5.0|
| 160644|        5.0|
| 172909|        5.0|
|   5490|        5.0|
| 172583|        5.0|
|   2972|        5.0|
|   2196|        5.0|
|  25887|        5.0|
| 118834|        5.0|
|   3795|        5.0|
| 156025|        5.0|
|   3941|        5.0|
| 120130|        5.0|
|    626|        5.0|
|   1349|        5.0|
|  26928|        5.0|
| 122092|        5.0|
|  40491|        5.0|
|  71268|        5.0|
| 167064| 

task 4

In [30]:
ratings_and_tags = ratings.alias("rating").join(tags.alias("tag"), on=["userId", "movieId"], how="inner")
delta = ratings_and_tags.select((functions.col("tag.timestamp") - functions.col("rating.timestamp")).alias("delta"))
display(delta.select(functions.avg(functions.col("delta"))).collect()[0])

Row(avg(delta)=26243727.372266974)

task 5

In [44]:
print('Average of average grades')
average_ratings_by_user = ratings.drop('movieId', 'timestamp').groupBy('userId').agg({"rating": "avg"}).orderBy('avg(rating)', ascending=False)
display(average_ratings_by_user.select(functions.avg(functions.col("avg(rating)"))).collect()[0])

Average of average grades


Row(avg(avg(rating))=3.6572223377474016)